In [ ]:
# 0. Importing Essential Libraries
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
import numpy as np  # 수치 계산을 위한 라이브러리
import re  # 정규표현식 처리에 사용 (예: 번호판 파싱 등)
import sys  # 시스템 관련 기능 제공 (예: 경로 설정 등)
import os  # 운영체제와의 상호작용 (예: 파일 경로 처리 등)

from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 특성 스케일링 도구
from sklearn.impute import SimpleImputer  # 결측값 처리 도구
from sklearn.model_selection import KFold, StratifiedKFold  # 교차검증 전략
from sklearn.ensemble import GradientBoostingRegressor  # 앙상블 기반 회귀 모델
from sklearn.linear_model import Ridge  # 릿지 회귀 모델 (선형 모델)
from sklearn.metrics import mean_squared_error  # 회귀 평가 지표 (평균제곱오차)
# 1. Data Loading and Initial Preparation
# 0. Importing Essential Libraries
import numpy as np
import pandas as pd
import re
import sys
from pathlib import Path

# Scikit-learn for preprocessing and model selection utilities
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# 1. 경로 설정 (JupyterLab 방식)
base_path = Path('/Users/jangyeeun/Desktop/Kaggle01/russian-car-plates-prices-prediction/train.csv') 
train_path = base_path / 'train.csv'
test_path = base_path / 'test.csv'
supplemental_path = base_path / 'supplemental_english.py'

# 2. 데이터 불러오기
print("\n데이터 불러오는 중")
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
print(f"Train data shape: {train.shape}")
print(f"Test data shape: {test.shape}")

# 4. supplemental_english.py 불러오기
sys.path.append(str(base_path))
import supplemental_english as supp
print("supplemental_english.py 로드 완료")

# 5. Train/Test 합치기
train['is_train'] = 1
test['is_train'] = 0
df = pd.concat([train, test], ignore_index=True)
print(f"Combined DataFrame shape: {df.shape}")
print("Train과 Test 데이터 병합 완료")

df
## <mark>📌 왜 Train과 Test를 합칠까?</mark>

# 1. 동일한 전처리 적용
# - `train`과 `test`를 따로 처리하면, **같은 값인데도 다르게 인코딩되거나**, **스케일 기준이 달라질 수 있음**
# - 따라서 **합쳐서 처리 후, 나중에 다시 분리하는 게 가장 안전**

# 2. 일관된 Feature 생성
#   - `test`까지 포함해서 만들면, **더 일반화된 특징**을 얻을 수 있기 때문

# 3. 추가 컬럼을 편하게 생성하기 위해
# - 파생변수를 만들 때,`train`, `test` 나눠서 처리하는 것보다 **하나로 합쳐서 처리하면 더 효율적이기 때문**
# - Kaggle에 제출할 때는 파생변수를 제출하는 게 아니라 그걸 기반으로 만든 예측 결과만 제출하면 되기 때문에 괜찮음.

# >💡 'is_train' 컬럼은 왜 추가하는지?
# > 나중에 다시 train과 test로 구분하기 위해
# 2. Plate Component Extraction and Basic Feature Engineering
# 자동차 번호판 문자열에서 구성 요소 추출 함수 정의  
# 러시아 차량 번호판 형식: 문자-3자리 숫자-2자리 문자-지역 코드  
def extract_components(plate):
    """
    차량 번호판 문자열에서 첫 문자, 3자리 숫자, 마지막 두 문자, 지역 코드, 전체 문자 조합을 추출
    """
    match = re.match(r'^([A-Z])(\d{3})([A-Z]{2})(\d{2,3})$', plate)
    if match:
        first_letter = match.group(1)       # 첫 번째 문자
        number = match.group(2)             # 3자리 숫자
        last_letters = match.group(3)       # 마지막 두 문자
        region_code = match.group(4)        # 지역 코드 (2~3자리)
        full_letters = first_letter + last_letters  # 전체 문자 조합
        return first_letter, number, last_letters, region_code, full_letters
    return None, None, None, None, None

# 번호판 구성 요소 추출하여 새로운 컬럼 생성
print("번호판 구성 요소 추출 중...")
df[['pre_lettre', 'numero', 'second_lettre', 'code_region', 'lettre_complet']] = \
    pd.DataFrame(df['plate'].apply(extract_components).tolist(), index=df.index)
print("번호판 구성 요소 추출 완료")
df
## <mark> 💙 plate - 문자 기반 파생 컬럼</mark>

# | 컬럼명            | 의미                         |
# |------------------|------------------------------|
# | `pre_lettre`     | 번호판 앞부분의 **첫 문자**      |
# | `numero`         | 문자 사이에 있는 **숫자**        |
# | `second_lettre`  | 번호판 뒷부분의 **마지막 문자**   |
# | `code_region`    | 번호판에 표시된 **지역 코드**     |
# | `lettre_complet` | 번호판의 전체 **문자 조합**       |
# 'date' 컬럼을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'])
print("'date' 컬럼을 datetime 형식으로 변환 완료.")

# 날짜에서 다양한 시간 관련 특성 추출 함수
def enrich_date_features(df):
    """
    날짜(datetime) 컬럼에서 다양한 시간 기반 특성과 주기성을 나타내는 변수를 추출합니다.
    """
    # 기본 시간 특성 추출: 연도, 월, 일, 요일, 연중 주차, 분기, 기준일로부터의 일수, 주말 여부, 요일 이름
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['date'].dt.quarter
    df['total_days'] = (df['date'] - df['date'].min()).dt.days
    df['is_weekend'] = df['date'].dt.dayofweek.isin([5, 6]).astype(int)
    df['day_name'] = df['date'].dt.day_name()

    # 주기성을 나타내는 특성 추가 (사인/코사인 변환): 요일, 일, 월
    # 모델이 시간 주기를 인식할 수 있도록 도와줌
    df['weekday_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    return df

# 날짜 관련 특성 추가 수행
df = enrich_date_features(df)
print("날짜 특성을 기본 + 주기적 특성으로 확장 완료.")
df
df.columns
## <mark> 💙 date 기반 파생 컬럼 </mark>

| 컬럼명           | 설명                                                    |
|------------------|---------------------------------------------------------|
| `year`           | 날짜의 **연도** (예: 2024)                              |
| `month`          | 날짜의 **월** (1~12)                                    |
| `day`            | 날짜의 **일** (1~31)                                    |
| `day_of_week`    | **요일 숫자** (월:0 ~ 일:6)                              |
| `week_of_year`   | 해당 날짜가 **1년 중 몇 번째 주**인지                   |
| `quarter`        | 해당 날짜의 **분기** (1~4분기)                           |
| `total_days`     | 기준일로부터의 **누적 일수** 또는 날짜 간격              |
| `is_weekend`     | **주말 여부** (토/일이면 1, 평일이면 0)                 |
| `day_name`       | 요일 이름 (예: Monday, Tuesday 등)                      |
| `weekday_sin`    | 요일 정보를 **사인 함수**로 변환한 값 (주기성 보존용)    |
| `weekday_cos`    | 요일 정보를 **코사인 함수**로 변환한 값                 |
| `day_sin`        | 날짜(1~31)를 **사인 함수**로 변환한 값                  |
| `day_cos`        | 날짜(1~31)를 **코사인 함수**로 변환한 값                |
| `month_sin`      | 월(1~12)을 **사인 함수**로 변환한 값                    |
| `month_cos`      | 월(1~12)을 **코사인 함수**로 변환한 값                  |
## <mark>📌 왜 사인·코사인 변환을 사용할까?</mark>
>- 사인(sin)·코사인(cos) 변환은  <b>날짜나 시간처럼 주기성(cyclical) </b>이 있는 데이터를 수치화할 때 사용되는 기법
>- 일반적인 숫자처럼 다루면 모델이 주기성을 제대로 인식하지 못해서 성능이 떨어질 수 있기 때문에 이를 해결하려고 사용
>- 2 * np.pi * df['day_of_week'] / 7  은 <b>요일 숫자(0~6)</b>를 <b>원의 각도(라디안)</b>로 변환하는 과정
>- 예시
| 요일 | 요일 이름 | θ (라디안)           | cos(θ) | sin(θ) | 좌표 (x, y)        |
| -- | ----- | ----------------- | ------ | ------ | ---------------- |
| 0  | 월요일   | $0$               | 1.000  | 0.000  | (1.000, 0.000)   |
| 1  | 화요일   | $\frac{2\pi}{7}$  | 0.623  | 0.782  | (0.623, 0.782)   |
| 2  | 수요일   | $\frac{4\pi}{7}$  | -0.222 | 0.975  | (-0.222, 0.975)  |
| 3  | 목요일   | $\frac{6\pi}{7}$  | -0.901 | 0.434  | (-0.901, 0.434)  |
| 4  | 금요일   | $\frac{8\pi}{7}$  | -0.901 | -0.434 | (-0.901, -0.434) |
| 5  | 토요일   | $\frac{10\pi}{7}$ | -0.222 | -0.975 | (-0.222, -0.975) |
| 6  | 일요일   | $\frac{12\pi}{7}$ | 0.623  | -0.782 | (0.623, -0.782)  |


>- sin과 cos를 둘 다 사용하는 이유는 주기적 특성을 2차원 좌표계에 나타내기 위함(하나만 쓰면 정보가 모자라서 주기성을 완전히 표현할 수 없음)
>- cos(각도)는 x좌표,sin(각도)는 y좌표
>- 예시
월요일 = 0 일 때,2π⋅0/7=0 따라서, cos(0)=1,sin(0)=0, 좌표(1,0)
>- (x,y)=(cos(θ),sin(θ))
from IPython.display import Image, display
display(Image(filename="sincos.png", width=400))
# 3. Extracting Information from Supplemental Data
# supplemental_english.py 파일에서 REGION_CODES를 명시적으로 불러오기
# REGION_CODES 딕셔너리 구조에 프로그래밍적으로 접근할 수 있도록 함
file_path = '/Users/jangyeeun/Desktop/캐글/russian-car-plates-prices-prediction/supplemental_english.py'
with open(file_path, 'r', encoding='utf-8') as file:
    python_content = file.readlines()

# REGION_CODES 딕셔너리 추출
# 이 로직은 Python 파일에서 REGION_CODES 변수를 파싱함
# 파싱 : 복잡한 텍스트(문자열)를 구조화된 데이터로 해석해서 필요한 정보를 추출하는 과정
region_start = [i for i, line in enumerate(python_content) if "REGION_CODES = {" in line][0]
bracket_count = 0
region_end = None

for i in range(region_start, len(python_content)):
    line = python_content[i]
    bracket_count += line.count("{") - line.count("}")  # 중괄호 열기/닫기 개수 추적
    if bracket_count == 0:
        region_end = i
        break
region_lines = python_content[region_start:region_end+1]

# 딕셔너리를 평탄화하여 지역 코드에 대한 DataFrame 생성
# 평탄화(flattening): 중첩된 구조(딕셔너리 내부의 리스트)를 1차원 리스트로 변환하여 정리하는 작업

region_codes = []
for line in region_lines[1:]:  # 'REGION_CODES = {' 라인은 건너뜀
    if ":" in line:
        key, value = map(str.strip, line.split(":", 1))  # 지역명(key)과 지역 코드 리스트(value)를 문자열로 분리하고 공백 제거
        value = value.rstrip(",").replace("[", "").replace("]", "").replace("\"", "").split(",")  # 리스트에서 불필요한 기호 제거 후 분할
        region_codes.extend([(key.strip('"'), code.strip()) for code in value if code.strip()])  # 지역명과 코드 쌍을 튜플로 저장, 빈 문자열 제외

# 모든 지역명과 지역코드를 튜플 형태로 리스트에 저장한 것을 DataFrame으로 변환
region_codes_df = pd.DataFrame(region_codes, columns=["region_name", "region_code"])
region_codes_df['region_code'] = region_codes_df['region_code'].str.strip()  # 코드 값의 앞뒤 공백 제거
print("supplemental 파일에서 REGION_CODES 파싱 완료.")


# 'code_region' 기준으로 원본 데이터프레임과 지역명을 병합
df = df.merge(region_codes_df, left_on='code_region', right_on='region_code', how='left')
# 병합 후 중복된 'region_code' 컬럼 제거
df.drop(columns=['region_code'], inplace=True)
print("지역 코드를 기준으로 지역명 병합 완료.")

df.columns

## 💙 파생피처 중간 정리
>- 기본 컬럼 : 'id', 'plate', 'date', 'price'
>- 트레이닝/테스트셋 구분 : 'is_train'
>- plate - 문자 : 'pre_lettre', 'numero','second_lettre', 'code_region', 'lettre_complet'
>- date : 'year', 'month',
       'day', 'day_of_week', 'week_of_year', 'quarter', 'total_days',
       'is_weekend', 'day_name', 'weekday_sin', 'weekday_cos', 'day_sin',
       'day_cos', 'month_sin', 'month_cos'
>- 지역코드 - 지역 이름 : 'region_name'
# 'supp' 모듈의 REGION_CODES를 직접 사용하여 숫자 코드 생성

def get_region_code_numeric(row):
    """region code를 REGION_CODES 딕셔너리에서 첫 번째 값으로 변환하여 숫자화"""
    code_region = str(row['code_region'])  # 문자열로 변환하여 조회
    for region, codes in supp.REGION_CODES.items():
        if code_region in codes:
            return int(codes[0])  # 해당 지역의 첫 번째 코드 반환
    return -1  # 해당 없음 시 -1 반환

df['region_code_numeric'] = df.apply(get_region_code_numeric, axis=1)
print("'region_code_numeric' 특성 생성 완료.")
# 정부 관련 정보 컬럼 초기화
df['is_government'] = 0
df['government_agency'] = None
df['forbidden_to_buy'] = False
df['road_advantage'] = False
df['significance_level'] = 0

# 'GOVERNMENT_CODES'를 활용한 정부 차량 번호판 정보 추출 함수
def get_government_info(row):
    """
    번호판 구성 요소와 GOVERNMENT_CODES 딕셔너리를 이용해 정부 차량 여부 및 정보를 반환
    """
    # 결측값이 있는 경우 예외 처리
    if pd.isna(row['pre_lettre']) or pd.isna(row['numero']) or pd.isna(row['code_region']):
        return 0, None, False, False, 0

    first_letter = row['pre_lettre']
    numbers = int(row['numero']) if pd.notna(row['numero']) else -1
    region_code = row['code_region']

    # 정의된 정부 차량 코드 탐색
    for (letters, (start, end), code), (agency, forbidden, advantage, significance) in supp.GOVERNMENT_CODES.items():
        # 번호판이 정부 차량 패턴과 일치하는지 확인
        if first_letter == letters[0] and region_code == code and start <= numbers <= end:
            return 1, agency, bool(forbidden), bool(advantage), significance

    return 0, None, False, False, 0  # 해당 없음 시 기본값 반환

# 정부 차량 정보를 각 행에 적용하여 새로운 특성 생성
print("정부 차량 번호판 정보 추출 중...")
govt_info = df.apply(get_government_info, axis=1)
df['is_government'] = [info[0] for info in govt_info]
df['government_agency'] = [info[1] for info in govt_info]
df['forbidden_to_buy'] = [info[2] for info in govt_info]
df['road_advantage'] = [info[3] for info in govt_info]
df['significance_level'] = [info[4] for info in govt_info]
print("정부 차량 관련 특성 생성 완료.")

# 열 이름 변경: 가독성 및 일관성 향상
df.rename(columns={
    'pre_lettre': 'first_letter',
    'second_lettre': 'last_letters',
    'lettre_complet': 'full_letters',
    'numero': 'numbers',
    'code_region': 'region_code_original'  # 'region_code_numeric'와 혼동 방지를 위해 이름 변경
}, inplace=True)
print("열 이름 변경 완료.")

# 'numbers' 컬럼을 정수형으로 변환 (에러는 NaN으로 처리 후 0으로 대체)
# 이후 수치 비교 및 계산을 위해 필수
df['numbers'] = pd.to_numeric(df['numbers'], errors='coerce').fillna(0).astype(int)
print("'numbers' 컬럼 정수형으로 변환 완료.")
#Frequency Encoding for the 'numbers' feature
# This replaces the number with its frequency of occurrence in the dataset.
freq_table = df['numbers'].value_counts().reset_index()
freq_table.columns = ['numbers', 'n']
freq_table['freq_enc'] = freq_table['n'] / freq_table['n'].sum()
freq_table['log_freq_enc'] = np.log1p(freq_table['freq_enc']) # Log transform for potential skewed distribution

# Merge frequency encodings back to the main DataFrame
df = df.merge(freq_table[['numbers', 'freq_enc', 'log_freq_enc']], 
              on='numbers', how='left')
df.rename(columns={'freq_enc': 'numbers_freq_enc', 
                  'log_freq_enc': 'numbers_log_freq_enc'}, inplace=True)
print("Applied Frequency Encoding to 'numbers' feature.")

# Target Encoding (Mean Encoding) for categorical features
# This technique replaces a categorical value with the mean of the target variable
# for that category. It's crucial to perform this only on the training data
# to avoid data leakage from the test set.
train_data = df[df['is_train'] == 1].copy()

# For regions: calculate mean price for each region name
region_mean_price = train_data.groupby('region_name')['price'].mean().reset_index()
region_mean_price.columns = ['region_name', 'region_mean_price']
df = df.merge(region_mean_price, on='region_name', how='left')
print("Target encoded 'region_name' with 'region_mean_price'.")

# For first letter: calculate mean price for each first letter
first_letter_mean_price = train_data.groupby('first_letter')['price'].mean().reset_index()
first_letter_mean_price.columns = ['first_letter', 'first_letter_mean_price']
df = df.merge(first_letter_mean_price, on='first_letter', how='left')
print("Target encoded 'first_letter' with 'first_letter_mean_price'.")

# For last letters: calculate mean price for each last letters combination
last_letters_mean_price = train_data.groupby('last_letters')['price'].mean().reset_index()
last_letters_mean_price.columns = ['last_letters', 'last_letters_mean_price']
df = df.merge(last_letters_mean_price, on='last_letters', how='left')
print("Target encoded 'last_letters' with 'last_letters_mean_price'.")

# Logarithmic transformation of the target variable 'price'
# This is a common practice in regression to make the target distribution more normal
# and reduce the impact of outliers, improving model performance.
df['log_price'] = np.log1p(df['price'])
print("Applied logarithmic transformation (log1p) to 'price' to create 'log_price'.")

# --- Newly Added Features for enhanced modeling ---

# Number Length and Uniqueness:
df['number_length'] = df['numbers'].apply(lambda x: len(str(x))) # Length of the numeric part
df['is_single_digit'] = (df['number_length'] == 1).astype(int) # Binary flag for single-digit numbers
print("Added 'number_length' and 'is_single_digit' features.")

# Frequency of letter + region combinations:
# This captures the popularity or rarity of specific plate patterns within regions.
df['letters_region'] = df['full_letters'] + "_" + df['region_code_original'].astype(str)
freq_lr = df['letters_region'].value_counts(normalize=True).to_dict()
df['letters_region_freq'] = df['letters_region'].map(freq_lr)
print("Calculated 'letters_region_freq' for letter-region combinations.")

# Relative Prestige Ranking:
# Convert prestige score to a rank, normalized between 0 and 1.
# This gives a relative measure of prestige across all plates.
from scipy.stats import rankdata
df['prestige_rank'] = rankdata(df['prestige_score'].astype(int), method='average') / len(df)
print("Created 'prestige_rank' based on 'prestige_score'.")

# Interaction Features:
df['letter_number_combo'] = df['full_letters'] + "_" + df['numbers'].astype(str)
# Interaction between 'is_government' and 'prestige_score'
df['is_gov_and_prestige'] = df['is_government'] * df['prestige_score'].astype(int)
print("Added 'letter_number_combo' and 'is_gov_and_prestige' interaction features.")

# Similarity with Known Plates (Textual Embedding using CountVectorizer):
# This attempts to capture patterns in letter sequences.
from sklearn.feature_extraction.text import CountVectorizer

# Using character n-grams to capture patterns like 'AA', 'AB', 'BA'
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,2))
# Apply to 'full_letters' (e.g., 'XAA', 'TMM')
letter_features = vectorizer.fit_transform(df['full_letters'].fillna(''))
# Note: 'letter_features' is a sparse matrix and needs to be integrated into the
# modeling pipeline if directly used. For now, it's generated for demonstration.
print(f"Generated textual features for 'full_letters' using CountVectorizer. Shape: {letter_features.shape}")

# Finer Geography:
# Flag common premium regions (e.g., major cities/oblasts) as a binary feature.
premium_regions = ['Moscow', 'Saint Petersburg', 'Moscow Oblast']
df['is_premium_region'] = df['region_name'].isin(premium_regions).astype(int)
print("Created 'is_premium_region' feature for major economic centers.")

# End of Feature Engineering section
print("\nFeature engineering complete. DataFrame is ready for model training.")
print(f"Final DataFrame shape after feature engineering: {df.shape}")
# 4. Handling Missing Values and Categorizing Government Agencies
# 'government_agency'의 결측값을 "Non-governmental"로 대체
df['government_agency'] = df['government_agency'].fillna('Non-governmental')
print("Missing 'government_agency' values filled with 'Non-governmental'.")

# 정부 기관을 보다 일반적인 그룹으로 분류하는 함수
# 이는 'government_agency' 컬럼의 높은 범주 수를 줄이기 위함입니다.
def categorize_agency(agency):
    """
    특정 정부 기관명을 더 넓고 일반적인 그룹으로 분류하여
    피처를 단순화합니다.
    """
    if agency == 'Non-governmental':
        return 'Non-governmental'
    elif 'President' in agency:
        return 'Presidential'
    elif 'Police' in agency.lower() or 'Internal Affairs' in agency:
        return 'Police/Security'
    elif 'Government' in agency:
        return 'Government'
    elif 'Military' in agency or 'Army' in agency or 'Defense' in agency:
        return 'Military'
    elif 'Federal' in agency:
        return 'Federal Services'
    elif 'Judge' in agency or 'Court' in agency or 'Justice' in agency or 'prosecutor' in agency.lower():
        return 'Judicial'
    elif 'Administration' in agency:
        return 'Administration'
    else:
        return 'Other Governmental'

# 분류 함수를 적용해 새로운 'agency_category' 피처 생성
df['agency_category'] = df['government_agency'].apply(categorize_agency)
print("Government agencies categorized into broader groups.")

# 각 기관 그룹에 대해 원-핫 인코딩으로 이진 변수 생성
# 범주형 변수를 모델에 적합한 수치형 변수로 변환합니다.
agency_dummies = pd.get_dummies(df['agency_category'], prefix='agency')
df = pd.concat([df, agency_dummies], axis=1)
print("One-hot encoded 'agency_category' into binary features.")

# (학습 데이터에 한해서) 기관 그룹별 평균 가격을 계산해 인사이트 도출
if 'price' in df.columns:  # 'price' 컬럼이 존재하는지 확인 후 계산
    agency_price = df[df['is_train'] == 1].groupby('agency_category')['price'].agg(['mean', 'count']).sort_values('mean', ascending=False)
    print("\nAverage price per agency category (Training Data):")
    print(agency_price)
# 5. Advanced Plate Feature Creation
# 'numbers' 컬럼의 결측값이 있을 경우 대비하여 다시 0으로 채움
df['numbers'] = df['numbers'].fillna(0)

# 'full_letters'에서 반복되는 문자가 있는지 확인 (예: 'AAA', 'XXX')
# 이런 패턴은 선호되는 특징일 수 있음
df['has_repeated_letters'] = df['full_letters'].str.replace(r'(.)(?=.*\1)', '', regex=True).str.len() < df['full_letters'].str.len()
print("Created 'has_repeated_letters' feature.")

# 'numbers'에서 반복되는 숫자가 있는지 확인 (예: '111', '777')
# 이런 숫자 패턴은 '아름다운' 또는 '명예로운' 숫자로 간주됨
df['has_repeated_numbers'] = df['numbers'].apply(
    lambda n: bool(re.search(r'(\d)\1', f"{int(n):03d}")) # 3자리 숫자 형식으로 변환 (예: 7 -> 007)
)
print("Created 'has_repeated_numbers' feature.")

# 연속된 숫자가 있는지 확인 (예: '123', '987')
# 이 역시 명예를 나타내는 패턴일 수 있음
df['has_sequential_numbers'] = df['numbers'].apply(
    lambda n: bool(re.search(r'123|234|345|456|567|678|789|987|876|765|654|543|432|321', f"{int(n):03d}"))
)
print("Created 'has_sequential_numbers' feature.")

# 거울 숫자(예: '121', '303') 혹은 팰린드롬 숫자(예: '111')인지 확인
# 이런 패턴도 특별한 의미를 가질 수 있음
df['has_mirror_numbers'] = df['numbers'].apply(
    lambda n: (str(int(n))[0] == str(int(n))[-1]) or (str(int(n)) == str(int(n))[::-1])
)
print("Created 'has_mirror_numbers' feature.")

# 명예로운 글자 조합 리스트 정의 (예: 'AAA', 'XXX' 등 특정 조합)
prestigious_letter_series = ["AAA", "MMM", "EEE", "KKK", "OOO", "PPP", "CCC", "TTT", "XXX"]
df['is_beautiful_series'] = df['full_letters'].isin(prestigious_letter_series)
print("Created 'is_beautiful_series' feature based on prestigious letter combinations.")

# 명예로운 숫자 조합 리스트 정의 (예: 단일 숫자, 3자리 반복 숫자, 100 단위 등)
prestigious_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 111, 222, 333, 444, 555, 666, 777, 888, 999,
                       100, 200, 300, 400, 500, 600, 700, 800, 900, 7]  # 7은 행운의 숫자로 자주 간주됨
df['is_prestigious_number'] = df['numbers'].isin(prestigious_numbers)
print("Created 'is_prestigious_number' feature based on specific prestigious number patterns.")

# 글자의 복잡도를 고유 문자 수로 계산
# 복잡도가 낮으면 (예: 'AAA') 단순하고 명예로운 것으로 판단 가능
df['letter_complexity'] = df['full_letters'].apply(
    lambda x: len(set(x)) if pd.notnull(x) else 0
)
print("Calculated 'letter_complexity' feature.")

# 여러 명예 관련 피처를 가중합하여 종합적인 명예 점수 생성
# 다양한 신호를 하나의 숫자 점수로 통합함
df['prestige_score'] = (
    (df['is_beautiful_series'].astype(int) * 3) +  # 아름다운 글자 조합에 높은 가중치 부여
    (df['is_prestigious_number'].astype(int) * 2) +  # 명예로운 숫자에 중간 가중치 부여
    (df['has_repeated_letters'].astype(int) * 1) +
    (df['has_repeated_numbers'].astype(int) * 1) +
    (df['has_sequential_numbers'].astype(int) * 1) +
    (df['has_mirror_numbers'].astype(int) * 1) +
    (df['significance_level'].fillna(0))  # 정부 중요도 점수 포함 (결측치는 0으로 처리)
)
print("Calculated 'prestige_score' by combining various prestige indicators.")

# 'prestige_score'를 범주형 타입으로 변환하여 모델 또는 시각화에 활용 가능하게 함
df['prestige_score'] = df['prestige_score'].astype('category')
print("Converted 'prestige_score' to categorical type.")
# 6. Encoding Categorical Variables and Advanced Feature Interactions


# 'numbers' 피처에 대해 빈도 인코딩 수행
# 데이터셋 내에서 숫자의 발생 빈도로 숫자를 대체함
freq_table = df['numbers'].value_counts().reset_index()
freq_table.columns = ['numbers', 'n']
freq_table['freq_enc'] = freq_table['n'] / freq_table['n'].sum()
freq_table['log_freq_enc'] = np.log1p(freq_table['freq_enc']) # 분포 왜곡 가능성을 줄이기 위한 로그 변환

# 빈도 인코딩 결과를 원본 DataFrame에 병합
df = df.merge(freq_table[['numbers', 'freq_enc', 'log_freq_enc']], 
              on='numbers', how='left')
df.rename(columns={'freq_enc': 'numbers_freq_enc', 
                  'log_freq_enc': 'numbers_log_freq_enc'}, inplace=True)
print("‘numbers’ 피처에 빈도 인코딩 적용 완료.")

# 범주형 피처에 대해 타겟 인코딩(평균 인코딩) 수행
# 각 카테고리에 대해 타겟 변수의 평균값으로 대체하는 기법.
# 데이터 누수를 막기 위해 반드시 학습 데이터에만 적용해야 함.
train_data = df[df['is_train'] == 1].copy()

# 지역별 평균 가격 계산
region_mean_price = train_data.groupby('region_name')['price'].mean().reset_index()
region_mean_price.columns = ['region_name', 'region_mean_price']
df = df.merge(region_mean_price, on='region_name', how='left')
print("‘region_name’에 대해 타겟 인코딩(‘region_mean_price’) 적용 완료.")

# 첫 글자별 평균 가격 계산
first_letter_mean_price = train_data.groupby('first_letter')['price'].mean().reset_index()
first_letter_mean_price.columns = ['first_letter', 'first_letter_mean_price']
df = df.merge(first_letter_mean_price, on='first_letter', how='left')
print("‘first_letter’에 대해 타겟 인코딩(‘first_letter_mean_price’) 적용 완료.")

# 마지막 글자 조합별 평균 가격 계산
last_letters_mean_price = train_data.groupby('last_letters')['price'].mean().reset_index()
last_letters_mean_price.columns = ['last_letters', 'last_letters_mean_price']
df = df.merge(last_letters_mean_price, on='last_letters', how='left')
print("‘last_letters’에 대해 타겟 인코딩(‘last_letters_mean_price’) 적용 완료.")

# 타겟 변수 'price'에 로그 변환 적용
# 회귀 모델에서 타겟 분포를 정규분포에 가깝게 만들고 이상치 영향 감소 목적
df['log_price'] = np.log1p(df['price'])
print("‘price’에 로그 변환(log1p) 적용하여 ‘log_price’ 생성 완료.")
# --- 모델링 향상을 위한 신규 피처 추가 ---

# 숫자 길이 및 단일 숫자인지 여부
df['number_length'] = df['numbers'].apply(lambda x: len(str(x))) # 숫자 부분 길이
df['is_single_digit'] = (df['number_length'] == 1).astype(int) # 한 자리 숫자인지 여부 (이진 플래그)
print("‘number_length’와 ‘is_single_digit’ 피처 추가 완료.")

# 글자 + 지역 조합 빈도
# 특정 지역 내 차량 번호판 패턴의 인기 또는 희소성을 반영
df['letters_region'] = df['full_letters'] + "_" + df['region_code_original'].astype(str)
freq_lr = df['letters_region'].value_counts(normalize=True).to_dict()
df['letters_region_freq'] = df['letters_region'].map(freq_lr)
print("‘letters_region_freq’ (글자-지역 조합 빈도) 계산 완료.")

# 상대적 명성 순위 (prestige_score를 0~1 사이로 정규화한 순위)
# 전체 번호판 중 상대적 명성 정도를 나타냄
from scipy.stats import rankdata
df['prestige_rank'] = rankdata(df['prestige_score'].astype(int), method='average') / len(df)
print("‘prestige_score’를 기반으로 ‘prestige_rank’ 생성 완료.")

# 상호작용 피처 생성
df['letter_number_combo'] = df['full_letters'] + "_" + df['numbers'].astype(str)
# 'is_government'와 'prestige_score' 간 상호작용
df['is_gov_and_prestige'] = df['is_government'] * df['prestige_score'].astype(int)
print("‘letter_number_combo’ 및 ‘is_gov_and_prestige’ 상호작용 피처 추가 완료.")

# 알려진 번호판과의 유사도 (CountVectorizer를 활용한 문자 임베딩)
# 글자 시퀀스 패턴 포착 시도
from sklearn.feature_extraction.text import CountVectorizer

# 문자 n-gram (1~2글자 조합) 추출
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,2))
# 'full_letters' 컬럼에 적용 (예: 'XAA', 'TMM')
letter_features = vectorizer.fit_transform(df['full_letters'].fillna(''))
# 참고: 'letter_features'는 희소행렬로, 모델링 파이프라인에 통합하여 사용 가능
print(f"CountVectorizer로 ‘full_letters’ 문자 특징 생성 완료. 행렬 크기: {letter_features.shape}")

# 세분화된 지리 정보
# 주요 대도시 및 경제 중심지를 프리미엄 지역으로 표시하는 이진 피처
premium_regions = ['Moscow', 'Saint Petersburg', 'Moscow Oblast']
df['is_premium_region'] = df['region_name'].isin(premium_regions).astype(int)
print("주요 경제 중심지에 대해 ‘is_premium_region’ 피처 생성 완료.")

# 피처 엔지니어링 완료 메시지 출력
print("\n피처 엔지니어링 완료. 모델 학습을 위한 DataFrame 준비 완료.")
print(f"피처 엔지니어링 후 최종 DataFrame 크기: {df.shape}")

Model Training and Ensemble
from catboost import CatBoostRegressor

categorical_features = ['brand', 'model', 'fuel_type', 'transmission']  # 예시

model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    random_state=SEED,
    verbose=100
)

model.fit(X_train, y_train, cat_features=categorical_features)

인코딩 없이 CatBoost 사용하는 방식 ->  CatBoost 쓸거면 이거 사용
df.columns
# ---------------------------------------------
# ⚙️ 하이퍼파라미터 및 전역 설정
# ---------------------------------------------
SEED = 92       # 재현성 확보용 랜덤 시드
N_SPLITS = 10   # Stratified K-Fold 교차검증용 분할 수
TARGET = 'log_price'  # 예측할 타겟 변수

# ---------------------------------------------
# 🧹 학습 시 제외할 컬럼 목록
# ---------------------------------------------
DROP_COLS = [
    'id', 'plate', 'price', 'log_price', 'is_train',
    "is_number_000", "is_number_444", "is_number_222", "is_number_700", 
    "is_number_555", "quarter", "day_of_week", "is_weekend",
    "prestige_score", "is_number_300","is_number_333","is_number_400",
    'significance_level', 'numbers_log_freq_enc', 'is_gov_and_prestige'
]

# ---------------------------------------------
# 📊 범주형 및 수치형 변수 지정
# ---------------------------------------------
categorical_features = [
    'brand', 'model', 'fuel_type', 'transmission', 'color', 'region'
]

numeric_features = [
    'engine_size', 'mileage', 'year', 'power', 'doors', 'seats'
]

# ---------------------------------------------
# 🔄 전처리기 구성
# ---------------------------------------------
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


categorical_transformer = OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

numeric_transformer = 'passthrough'  # 수치형은 그대로 사용

preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('num', numeric_transformer, numeric_features)
])

# ---------------------------------------------
# 🧱 최종 파이프라인 (XGBoost 사용 예시)
# ---------------------------------------------
from catboost import CatBoostRegressor

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    random_state=SEED,
    verbose=100
    ))
])
## 1. TRAIN / TEST SPLIT
# 'is_train' 플래그를 기준으로 합쳐졌던 DataFrame을 원래의 학습용과 테스트용 데이터로 분리
train_df = df[df['is_train'] == 1].copy()  # .copy()를 사용하여 SettingWithCopyWarning 방지
test_df = df[df['is_train'] == 0].copy()

# 학습 데이터셋에서 특징(X)과 타겟(y)을 정의하고,
# 테스트 데이터셋에서는 예측에 사용할 특징(X_test)만 정의
# DROP_COLS에 명시된 컬럼이 존재하지 않아도 에러가 발생하지 않도록 errors='ignore' 사용
X = train_df.drop(columns=DROP_COLS, errors='ignore')  # 학습용 특징
y = train_df[TARGET].copy()                            # 타겟값
X_test = test_df.drop(columns=DROP_COLS, errors='ignore')  # 테스트용 특징

print("데이터를 학습용과 테스트용으로 분리했습니다.")
print(f"학습 특징(X) 데이터 형태: {X.shape}")
print(f"학습 타겟(y) 데이터 형태: {y.shape}")
print(f"테스트 특징(X_test) 데이터 형태: {X_test.shape}")
X.columns
import pandas as pd
import numpy as np

# 수치형 컬럼만 추출
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# 상관 행렬 계산
corr_matrix = df[numerical_cols].corr().abs()

# 상삼각 행렬만 남기기 (자기 자신 제외)
upper_tri = corr_matrix.where(~np.tril(np.ones(corr_matrix.shape)).astype(bool))

# 상관계수 0.85 이상인 컬럼들 추출
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]

print("다중공선성 의심 컬럼들:\n", to_drop)


## 2. AUTOMATIC COLUMN DETECTION (Enhanced)
# 이 섹션은 다양한 데이터 타입을 동적으로 처리하는 데 핵심적인 역할을 합니다.
# 숫자형, 불리언형, 범주형 컬럼을 자동으로 식별하고,
# 범주형 컬럼은 고유값 개수(카디널리티)에 따라 세분화하여
# 적절한 인코딩 전략을 적용할 수 있도록 합니다.

def detect_columns(X):
    """
    데이터 타입 및 카디널리티 기준으로 컬럼들을 분류합니다.
    각 컬럼 유형에 맞는 전처리 단계를 적용하는 데 활용됩니다.
    """
    bool_cols = [c for c in X.columns if X[c].dtype == 'bool']  # 불리언(boolean) 컬럼 식별
    num_cols = [c for c in X.columns if X[c].dtype.kind in 'if' and c not in bool_cols]  # 숫자형(int/float) 컬럼 식별 (불리언 제외)
    cat_cols = [c for c in X.columns if c not in num_cols + bool_cols]  # 나머지는 범주형(categorical) 컬럼으로 간주

    # 범주형 컬럼을 고유값 개수(카디널리티)에 따라 세분화
    # 카디널리티에 따라 최적의 인코딩 방식이 달라지므로 이를 고려함
    cat_low = [c for c in cat_cols if X[c].nunique() <= 20]      # 고유값이 20 이하 → One-Hot Encoding 적합
    cat_mid = [c for c in cat_cols if 20 < X[c].nunique() <= 200] # 고유값이 중간 수준 → Ordinal Encoding 적합
    cat_high = [c for c in cat_cols if X[c].nunique() > 200]     # 고유값이 매우 많음 → Target Encoding 적합

    print('\n컬럼 요약 ➜ 숫자형:', len(num_cols),
          '| 불리언:', len(bool_cols),
          '| 낮은 카디널리티 범주형:', len(cat_low),
          '| 중간 카디널리티 범주형:', len(cat_mid),
          '| 높은 카디널리티 범주형:', len(cat_high))

    return num_cols, bool_cols, cat_low, cat_mid, cat_high

# 학습 데이터의 피처(X)에 대해 컬럼 분류 함수 적용
num_cols, bool_cols, cat_low, cat_mid, cat_high = detect_columns(X)
## 3. PREPROCESSING PIPELINE (Comprehensive and Flexible)
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 다양한 데이터 타입에 맞춰 적절한 전처리 방식을 병렬로 적용할 수 있도록 해주는 ColumnTransformer를 정의합니다.
# 이 구성은 모델 학습 전에 각 특성에 맞는 전처리를 안정적으로 수행할 수 있게 해줍니다.

preprocess = ColumnTransformer(
    transformers=[
        # 수치형 변수들은 특별한 변환 없이 그대로 사용 ('passthrough'는 아무 변환도 하지 않음을 의미)
        ('num', 'passthrough', num_cols),
        
        # 이진 변수 (True/False)는 이미 0과 1로 구성되어 있으므로 그대로 사용
        ('bool', 'passthrough', bool_cols),

        # 저중복 범주형 변수: One-Hot Encoding 사용
        # - 각 범주마다 새로운 열을 생성하여 0 또는 1의 값으로 표현
        # - handle_unknown='ignore' 설정으로 테스트 셋에서 새로운 범주가 등장해도 오류 없이 처리
        # - sparse_output=False는 밀집 배열로 출력하게 하여 이후 pandas나 numpy와의 연동을 쉽게 함
        ('low', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_low),

        # 중복 수가 중간 수준인 범주형 변수: Ordinal Encoding 사용
        # - 각 범주에 고유 정수값을 매겨서 숫자로 변환
        # - handle_unknown='use_encoded_value', unknown_value=-1 옵션으로 테스트 데이터에 존재하지 않는 범주가 나와도 예외 발생 방지
        ('mid', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_mid),

        # 고중복 범주형 변수: 원래는 Target Encoding을 사용했으나, 외부 패키지 의존성 제거를 위해 Ordinal Encoding으로 대체
        # - 높은 카디널리티(범주의 수)가 있더라도, 각 범주를 고유 정수로 매핑하여 처리
        # - unknown_value=-1 설정으로 테스트 데이터에서 새로운 범주가 나와도 안정적으로 처리 가능
        ('high', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_high)
    ],

    # 위에서 지정하지 않은 열들은 모두 삭제 (불필요한 정보가 모델에 들어가지 않도록 안전 조치)
    remainder='drop',

    # 병렬 처리로 속도 향상을 위해 CPU 모든 코어를 사용
    n_jobs=-1
)

print("\nPreprocessing pipeline (ColumnTransformer) defined.")
## 4. MODELS AND PARAMETERS (Optimized and Modular)
# 다양한 그래디언트 부스팅 회귀 모델에 대해 최적화된 하이퍼파라미터 설정
# 이 하이퍼파라미터들은 GridSearchCV, RandomizedSearchCV, 또는 Optuna와 같은 고급 최적화 기법을 통해 도출됨
from xgboost import XGBRegressor

# XGBoost 하이퍼파라미터
xgb_params = {
    'n_estimators': 1433,  # 트리 개수
    'max_depth': 12,  # 트리 최대 깊이
    'learning_rate': 0.01852160907217988,  # 학습률
    'subsample': 0.6786672470738663,  # 샘플링 비율 (row)
    'colsample_bytree': 0.46208650739218005,  # 피처 샘플링 비율 (column)
    'reg_alpha': 0.017519138973638618,  # L1 정규화
    'reg_lambda': 0.2839310763317462,  # L2 정규화
    'gamma': 0.0033995958574628547,  # 분할 최소 손실 감소 값
    'tweedie_variance_power': 1.0869464555654937,  # Tweedie 분포 파라미터 (비대칭 타깃에 유용)
    'objective': 'reg:tweedie',  # 회귀 목적 함수
    'n_jobs': -1,  # 멀티스레딩 사용
    'random_state': SEED  # 랜덤 시드 고정
}

from lightgbm import LGBMRegressor

# LightGBM 하이퍼파라미터
lgb_params = {
    'n_estimators': 999,  # 부스팅 round 수 (트리 개수)
    'max_depth': 11,  # 트리의 최대 깊이
    'learning_rate': 0.07607568555547708,  # 학습률
    'subsample': 0.6363036032688429,  # 학습 샘플의 비율 (row-wise sampling)
    'colsample_bytree': 0.5072021102992719,  # 각 트리에서 사용할 피처의 비율 (column-wise sampling)
    'min_child_samples': 97,  # 리프 노드가 가져야 하는 최소 데이터 수
    'reg_alpha': 0.16671454380081874,  # L1 정규화 파라미터
    'reg_lambda': 0.6455320711051608,  # L2 정규화 파라미터
    'n_jobs': -1,  # 모든 CPU 코어 사용
    'random_state': SEED  # 랜덤 시드 고정
}

# CatBoost 하이퍼파라미터
cat_params = {
    'iterations': 991,  # 트리 개수
    'depth': 10,  # 트리 깊이
    'learning_rate': 0.06462213707942074,  # 학습률
    'l2_leaf_reg': 1.9289204888270515,  # L2 정규화 (리프별)
    'subsample': 0.7213225292844163,  # 데이터 샘플링 비율
    'bagging_temperature': 0.4361642090192932,  # 샘플 다양성 조절
    'random_strength': 6.443179917768372,  # 분할 시 랜덤성 조절
    'min_data_in_leaf': 72,  # 리프 최소 샘플 수
    'loss_function': 'RMSE',  # 손실 함수 (회귀에 일반적인 RMSE)
    'verbose': 0,  # 훈련 로그 생략
    'random_state': SEED
}

# 사용할 모델들을 딕셔너리 형태로 정의 (추가 모델 쉽게 포함 가능)
models = {
    'XGB': XGBRegressor(**xgb_params),
    'LGBM': LGBMRegressor(**lgb_params),
    'CatBoost': CatBoostRegressor(**cat_params)
}
print("\n모델 및 최적 하이퍼파라미터 정의 완료.")

# 각 모델별 파이프라인 구성: 전처리(preprocessing) → 모델 학습
# 각 파이프라인은 학습 전에 필요한 데이터 전처리까지 포함
pipelines = {name: Pipeline(steps=[('prep', preprocess), ('model', model)]) for name, model in models.items()}
print("파이프라인 구성 완료: 전처리 -> 모델.")

## 5. SMAPE METRIC Definition
# SMAPE(Symmetric Mean Absolute Percentage Error)는 시계열 예측에서 자주 사용되는 지표이며,
# 실제값이 0인 경우에도 비교적 강건한 평가 지표입니다.
# 한 번 정의해두면 일관된 평가를 위한 기준이 됩니다.

def smape(y_true, y_pred):
    """
    SMAPE(Symmetric Mean Absolute Percentage Error)를 계산합니다.
    공식: (1/n) * Σ(|y_true - y_pred| / ((|y_true| + |y_pred|) / 2)) * 100
    이 지표는 y_true나 y_pred가 0일 수 있는 상황에서도 사용할 수 있도록 설계되었습니다.
    """
    # 실제값과 예측값의 절댓값 평균을 분모로 사용
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    # 실제값과 예측값 차이의 절댓값 (분자)
    diff = np.abs(y_true - y_pred)
    
    # 분모가 0인 경우를 처리하기 위한 배열 초기화
    # 분모가 0인 경우 해당 항의 SMAPE는 0으로 처리하여 NaN 또는 Inf 방지
    smape_term = np.zeros_like(diff, dtype=float)
    
    # 분모가 0이 아닌 인덱스만 선택하여 나눗셈 수행
    non_zero_denom = denominator != 0
    smape_term[non_zero_denom] = diff[non_zero_denom] / denominator[non_zero_denom]
    
    # 전체 평균을 내어 백분율로 반환
    return np.mean(smape_term) * 100

print("\nSMAPE 평가 지표가 정의되었습니다.")
## 6. STRATIFIED CROSS-VALIDATION (on the target)
# Stratified K-Fold 교차검증은 각 폴드에 타겟 변수의 분포가 고르게 나타나도록 해줍니다.
# 이는 타겟이 불균형하거나 특정 구간이 중요할 때 특히 유용합니다.

# 타겟 변수(log_price)를 구간으로 나눠 계층화(Stratification)를 위한 "층(strata)" 생성
# 회귀 문제를 분류 문제처럼 취급하여 K-Fold 분할 시 타겟 분포를 유지함
from sklearn.preprocessing import KBinsDiscretizer

y_bins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile') \
    .fit_transform(y.values.reshape(-1, 1)).astype(int).ravel()
print(f"\n타겟 변수 ('{TARGET}')가 {y_bins.max() + 1}개의 층으로 구간화되었습니다.")

# StratifiedKFold 객체 초기화 (shuffle=True로 무작위 셔플, random_state 고정)
kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# 각 모델의 OOF(out-of-fold) 예측값과 테스트셋 예측값을 저장할 딕셔너리 초기화
# OOF는 앙상블 가중치 계산 및 최종 CV 평가에 사용
# 테스트 예측값은 폴드별 예측을 평균내어 제출용으로 사용
oof_preds = {name: np.zeros(len(y)) for name in models}
test_preds = {name: np.zeros(len(X_test)) for name in models}
feature_importances = {}  # 모델별 중요 피처 저장용

print('\n===== 교차검증 기반 훈련 시작 =====')

# 각 모델에 대해 교차검증을 수행
for model_name, pipeline in pipelines.items():
    print(f"\n모델 훈련 시작: {model_name}...")
    try:
        # StratifiedKFold에서 각 폴드에 대해 반복
        for fold, (train_idx, val_idx) in enumerate(kf.split(X, y_bins), 1):
            print(f"  폴드 {fold:02d}/{N_SPLITS}")
            
            # 현재 폴드의 학습 데이터와 검증 데이터 분리
            X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            # 파이프라인 학습 (전처리 + 모델 학습 포함)
            pipeline.fit(X_tr, y_tr)

            # 현재 폴드의 검증 데이터에 대한 예측값 저장 (OOF 예측)
            oof_preds[model_name][val_idx] = pipeline.predict(X_val)
            
            # 테스트셋 예측값 누적 (폴드별 평균 내어 사용)
            test_preds[model_name] += pipeline.predict(X_test) / N_SPLITS

        # 전체 폴드에 대한 OOF 예측을 바탕으로 SMAPE 계산 (로그 스케일 복원 후)
        cv_smape = smape(np.exp(y), np.exp(oof_preds[model_name]))
        print(f'⮕  {model_name} 모델의 전체 CV SMAPE: {cv_smape:.2f}%')

        # 피처 중요도 추출 (지원하는 모델에 한해)
        if hasattr(pipeline['model'], 'feature_importances_'):
            # LightGBM, scikit-learn의 트리 기반 모델 등
            feature_importances[model_name] = pipeline['model'].feature_importances_

        elif hasattr(pipeline['model'], 'get_booster'):
            # XGBoost는 get_booster()로 내부 Booster 접근 가능
            feature_importances[model_name] = pipeline['model'].get_booster().get_score(importance_type='weight')
            # importance_type은 'weight', 'gain', 'cover' 등으로 지정 가능

        else:
            # 중요도 추출 불가한 모델 처리
            feature_importances[model_name] = None

    except Exception as e:
        print(f"{model_name} 모델 훈련 중 오류 발생: {str(e)}")
        continue  # 오류 발생 시 해당 모델 건너뛰고 다음 모델로
## 7. ENSEMBLE PREDICTIONS (Inverse Error Weighting)
# 1차 모델들 OOF 예측과 테스트 예측을 모아서 메타 학습용 데이터셋 생성
X_meta = np.column_stack([oof_preds[name] for name in models])
X_test_meta = np.column_stack([test_preds[name] for name in models])
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=10, shuffle=True, random_state=42)

meta_oof = np.zeros(len(y))
meta_test_fold = []

for train_idx, val_idx in kf.split(X_meta):
    X_tr, X_val = X_meta[train_idx], X_meta[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    meta_model = RidgeCV(alphas=np.logspace(-3, 3, 10), cv=3)
    meta_model.fit(X_tr, y_tr)
    
    meta_oof[val_idx] = meta_model.predict(X_val)
    meta_test_fold.append(meta_model.predict(X_test_meta))

# 평균을 취해 최종 테스트셋 예측
ensemble_preds_log = np.mean(meta_test_fold, axis=0)
ensemble_preds = np.exp(ensemble_preds_log)

# OOF 기준 SMAPE 측정
ensemble_smape = smape(np.exp(y), np.exp(meta_oof))
print(f"\n⮕ K-Fold Stacking Ensemble CV SMAPE: {ensemble_smape:.2f}%")
## 8. SUBMISSION FILE CREATION
# 최종 예측 결과를 Kaggle 제출용으로 준비하는 단계입니다.

# 모델 예측값에 클리핑(clipping)을 적용하여 비현실적인 값을 방지합니다.
# a_min=0은 예측값이 0보다 작아지지 않도록 하고,
# a_max=None은 상한 제한을 두지 않음을 의미합니다.
# 클리핑은 이상치나 모델 오류로 인한 극단값을 완화하는 데 도움이 됩니다.
ensemble_preds = np.clip(ensemble_preds, a_min=0, a_max=None)

# 제출 파일을 위한 DataFrame 생성: 'id'와 최종 'price' 예측값을 포함합니다.
submission = pd.DataFrame({
    'id': test_df['id'],  # 원본 테스트 데이터의 'id' 열을 그대로 사용
    'price': ensemble_preds
})

# 제출용 CSV 파일로 저장 (Kaggle 형식에 맞게 인덱스 제외)
submission.to_csv('0521submission_sample5.csv', index=False)
print('\n✅  제출 파일 "submission.csv" 이(가) 성공적으로 저장되었습니다.')
